# Skill 4: Internet and Websites Search using Bing API - Bing Chat Clone

In this notebook, we'll delve into the ways in which you can **boost your GPT Smart Search Engine with web search functionalities**, utilizing both Langchain and the Azure Bing Search API service.

As previously discussed in our other notebooks, **harnessing agents and tools is an effective approach**. We aim to leverage the capabilities of OpenAI's large language models (LLM), such as GPT-3.5 and its successors, to perform the heavy lifting of reasoning and researching on our behalf.

There are numerous instances where it is necessary for our Smart Search Engine to have internet access. For instance, we may wish to **enrich an answer with information available on the web**, or **provide users with up-to-date and recent information**, or **finding information on an specific public website**. Regardless of the scenario, we require our engine to base its responses on search results.

By the conclusion of this notebook, you'll have a solid understanding of the Bing Search API basics, including **how to create a Web Search Agent using the Bing Search API**, and how these tools can strengthen your chatbot. Additionally, you'll learn about Callbacks, another way  of **how to observe Agent Actions and their significance in bot applications**.

In [1]:
import os
import requests
from typing import Dict, List, Optional, Type
import asyncio
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup


from langchain import hub
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import BingSearchAPIWrapper

from common.callbacks import StdOutCallbackHandler
from common.prompts import BINGSEARCH_PROMPT

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string.replace("$","USD ")))

from dotenv import load_dotenv
load_dotenv("credentials.env")


True

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Introduction to Callback Handlers

**Callbacks**:

LangChain provides a callbacks system, another way to monitor/observe agent actions, that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks. You can subscribe to these events by using the callbacks argument available throughout the API. This argument is a list of handler objects.

**Callback handlers**:

CallbackHandlers are objects that implement the CallbackHandler interface, which has a method for each event that can be subscribed to. The CallbackManager will call the appropriate method on each handler when the event is triggered.

---

We will incorporate a handler for the callbacks, enabling us to observe the response as it streams and to gain insights into the Agent's reasoning process. This will prove incredibly valuable when we aim to stream the bot's responses to users and keep them informed about the ongoing process as they await the answer.

Our custom handler is in the folder `common/callbacks.py`. Go and take a look at it.


In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS, 
                      streaming=True, callback_manager=cb_manager)


### Creating a custom tool - Bing Search API tool

Langhain has already a pre-created tool called BingSearchAPIWrapper, however we are going to make it a bit better by using the results function instead of the run function, that way we not only have the text results, but also the title and link(source) of each snippet.

In [4]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "Searcher"
    description = "useful to search the internet.\n"
    args_schema: Type[BaseModel] = SearchInput

    k: int = 5
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        loop = asyncio.get_event_loop()
        results = await loop.run_in_executor(ThreadPoolExecutor(), bing.results, query, self.k)
        return results

### Creating another custom tool - WebFetcher: Visits a website and extracts the text
    You will need GPT-4 with a big context token size for this tool since the content of a website can be very lenghty

In [5]:
def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

def fetch_web_page(url: str) -> str:
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'}
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

In [6]:
web_fetch_tool = Tool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="useful to fetch the content of a url"
)

### Creating the Agent

Now, we create our OpenAI Tools type agent that uses our custom tools and our custom prompt `BING_PROMPT_PREFIX`. Check it out in `prompts.py`

In [7]:
# tools = [MyBingSearch(k=5), web_fetch_tool] # With GPT-4 you can add the web_fetch_tool

tools = [MyBingSearch(k=5)] # With GPT-3.5 

prompt = BINGSEARCH_PROMPT

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False, 
                               return_intermediate_steps=True)

In [8]:
agent_executor.tools

[MyBingSearch()]

Try some of the below questions, or others that you might like

In [9]:
# QUESTION = "Create a list with the main facts on What is happening with the oil supply in the world right now?"
# QUESTION = "How much is 50 USD in Euros and is it enough for an average hotel in Madrid?"
# QUESTION = "My son needs to build a pinewood car for a pinewood derbi, how do I build such a car?"
# QUESTION = "I'm planning a vacation to Greece, tell me budget for a family of 4, in Summer, for 7 days including travel, lodging and food costs"
# QUESTION = "Who won the 2023 superbowl and who was the MVP?"
QUESTION = """
compare the number of job opennings (provide the exact number), the average salary within 15 miles of Dallas, TX, for these ocupations:

- ADN Registerd Nurse 
- Occupational therapist assistant
- Dental Hygienist
- Graphic Designer


# Create a table with your findings. Place the sources on each cell.
# """

### Agent Actions/Observations during streaming

Streaming is an important UX consideration for LLM apps, and agents are no exception. Streaming with agents is made more complicated by the fact that it’s not just tokens of the final answer that you will want to stream, but you may also want to stream back the intermediate steps an agent takes.

The outputs also contain richer structured information inside of actions and steps, which could be useful in some situations, but can also be harder to parse.

At the end of Notebook 3 we learned that streaming can be simply achieve by doing this:

```python
for chunk in chain.stream({"question": QUESTION, "language": "English", "history":""}):
    print(chunk, end="", flush=True)
```

At the end of Notebook 6 we learned about the new astream_events API (beta).

```python
async for event in agent_with_chat_history.astream_events(
    {"question": QUESTION}, config=config, version="v1"):
```

Now we are going to achieve the same result of the astream_events API, by combining Callbacks with the astream() function:

    With Agents, we would need to parse the information contained on each streamed chunk since it contains a lot of information and also use the callback handler to stream the tokens.

In [10]:
async for chunk in agent_executor.astream({"question": QUESTION}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        # Uncomment if you need to have the information retrieve from the tool
        # for step in chunk["steps"]:
        #     print(f"Tool Result: `{step.observation}`")
        continue
    # Final result
    elif "output" in chunk:
        # No need to print the final output again since we would be streaming it as it is produced
        # print(f'Final Output: {chunk["output"]}') 
        continue
    else:
        raise ValueError()
    print("---")

Calling Tool: `Searcher` with input `{'query': 'job openings for ADN Registered Nurse within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'average salary for ADN Registered Nurse within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'job openings for Occupational therapist assistant within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'average salary for Occupational therapist assistant within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'job openings for Dental Hygienist within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'average salary for Dental Hygienist within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'job openings for Graphic Designer within 15 miles of Dallas, TX'}`
---
Calling Tool: `Searcher` with input `{'query': 'average salary for Graphic Designer within 15 miles of Dallas, TX'}`
---


#### Without showing the intermedite steps, just the final answer

In [11]:
QUESTION = "How much is 50 USD in Euros and is it enough for an average hotel in Madrid?"

try:
    response = agent_executor.invoke({"question":QUESTION})
except Exception as e:
    response = str(e)

Based on the current exchange rate, 50 USD is approximately 45 EUR [[1]](https://www.xe.com/en/currencyconverter/convert/?Amount=50&From=USD&To=EUR). 

As for the average hotel cost in Madrid, the average price for a hotel for one week in Madrid is approximately $625, which is around 565 EUR. This means that 50 USD is not enough to cover the average hotel cost for one week in Madrid [[2]](https://www.budgetyourtrip.com/hotels/spain/madrid-3117735).

If you have any other questions or need further assistance, feel free to ask!

In [12]:
printmd(response["output"])

Based on the current exchange rate, 50 USD is approximately 45 EUR [[1]](https://www.xe.com/en/currencyconverter/convert/?Amount=50&From=USD&To=EUR). 

As for the average hotel cost in Madrid, the average price for a hotel for one week in Madrid is approximately USD 625, which is around 565 EUR. This means that 50 USD is not enough to cover the average hotel cost for one week in Madrid [[2]](https://www.budgetyourtrip.com/hotels/spain/madrid-3117735).

If you have any other questions or need further assistance, feel free to ask!

## QnA to specific websites

There are several use cases where we want the smart bot to answer questions about a specific company's public website. There are two approaches we can take:

1. Create a crawler script that runs regularly, finds every page on the website, and pushes the documents to Azure Cognitive Search.
2. Since Bing has likely already indexed the public website, we can utilize Bing search targeted specifically to that site, rather than attempting to index the site ourselves and duplicate the work already done by Bing's crawler.

Below are some sample questions related to specific sites. Take a look:

In [13]:
# QUESTION = "information on how to kill wasps in homedepot.com"
QUESTION = "in target.com, find how what's the price of a Nesspresso coffee machine and of a Keurig coffee machine"
# QUESTION = "in microsoft.com, find out what is the latests news on quantum computing"
# QUESTION = "give me on a list the main points on the latest investor report from mondelezinternational.com"

In [14]:
async for chunk in agent_executor.astream({"question": QUESTION}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        # Uncomment if you need to have the information retrieve from the tool
        # for step in chunk["steps"]:
        #     print(f"Tool Result: `{step.observation}`")
        continue
    # Final result
    elif "output" in chunk:
        # No need to print the final output again since we would be streaming it as it is produced
        # print(f'Final Output: {chunk["output"]}') 
        continue
    else:
        raise ValueError()
    print("---")

Calling Tool: `Searcher` with input `{'query': 'Nesspresso coffee machine price site:target.com'}`
---
Calling Tool: `Searcher` with input `{'query': 'Keurig coffee machine price site:target.com'}`
---
The price of a Nespresso coffee machine and a Keurig coffee machine at Target.com are as follows:

### Nespresso Coffee Machines
1. **Nespresso Vertuo Next Coffee Maker and Espresso Machine by De'Longhi**:
   - Price: $139.99 [[1]](https://www.target.com/p/nespresso-vertuo-next-coffee-maker-and-espresso-machine-by-delonghi-gray/-/A-79332989)

2. **Nespresso VertuoPlus Deluxe Coffee Maker and Espresso Machine by De'Longhi**:
   - Price: $139.99 (price in cart) [[2]](https://www.target.com/p/nespresso-vertuo-plus-deluxe-coffee-maker-and-espresso-machine-by-delonghi-titan/-/A-82749274)

### Keurig Coffee Machines
1. **Keurig K-Supreme Plus Coffee Maker - Stainless Steel**:
   - Price: $219.99 [[3]](https://www.target.com/b/keurig/-/N-5uvm4)

2. **Keurig K-Iced Plus Single-Serve K-Cup Pod Co

# Summary

In this notebook, we learned how to create a Bing Chat clone using a clever prompt with specific search and formatting instructions. We also learned about combining the Callback Handlers with the agent stream() or astream() functions, to stream the response from the LLM while showing the intermediate steps.  

The outcome is an agent capable of conducting intelligent web searches and performing research on our behalf. This agent provides us with answers to our questions along with appropriate URL citations and links!

**Note**: as we have said before GPT-4 will be more accurate following instructions, hold more space for context, and provide better responses.

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.